In [2]:
# Data loading and validation
import pandas as pd 

#Load dataset
df=pd.read_csv("Health_performance.csv")

In [3]:
# Quick Validation
print(df.shape) # rows and columns 
print(df.info()) # Data types 
print(df.head()) # First 5 rows
print(df.isnull().sum()) # Missing values count 

(25000, 20)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Patient_ID                   25000 non-null  object 
 1   Gender                       25000 non-null  object 
 2   Age                          25000 non-null  int64  
 3   City                         25000 non-null  object 
 4   State                        25000 non-null  object 
 5   Department                   25000 non-null  object 
 6   Admission_Date               25000 non-null  object 
 7   Discharge_Date               25000 non-null  object 
 8   Length_of_Stay               25000 non-null  int64  
 9   Room_Type                    25000 non-null  object 
 10  Treatment_Type               25000 non-null  object 
 11  Treatment_Cost_INR           25000 non-null  int64  
 12  Insurance_Provider           25000 non-null  object 
 13  Insu

In [6]:
# Data Cleaning 
# Convert date columns 
df['Admission_Date']=pd.to_datetime(df['Admission_Date'],errors='coerce')
df['Discharge_Date']=pd.to_datetime(df['Discharge_Date'],errors='coerce')


In [7]:
# Create Age Groups
def categorize_age(age):
    if age < 18: return 'Child'
    elif age <60: return 'Adult'
    else: return 'Senior'
df['Age_Group'] = df['Age'].apply(categorize_age)


In [8]:
# Handling missing values
df = df.dropna() # drop the rows with missing values

In [9]:
# Export cleaned dataset
df.to_csv("Cleaned_Health_Performance.csv", index= False)

In [11]:
pip install pymysql 

Note: you may need to restart the kernel to use updated packages.


In [12]:
import pymysql 
import pandas as pd 

In [14]:
# connect to mysql 
connection = pymysql.connect(
    host="localhost",
    user="root",
    password="sravzlakna@8",
    database="health_performance"
)
cursor = connection.cursor()

In [15]:
# Load Cleaned CSV 
df= pd.read_csv("Cleaned_Health_Performance.csv")

In [16]:
# Insert Data into SQL table 
# Prepare SQL Insert Statement
insert_query = """
INSERT INTO Health_Data (
    Patient_ID, Gender, Age, City, State, Department, Admission_Date, Discharge_Date,
    Length_of_Stay, Room_Type, Treatment_Type, Treatment_Cost_INR, Insurance_Provider,
    Insurance_Coverage_Pct, Insurance_Payout_INR, Patient_Pay_INR, Revenue_INR,
    Outcome, Patient_Satisfaction_Rating, Year, Age_Group
) VALUES (
    %s, %s, %s, %s, %s, %s, %s, %s,
    %s, %s, %s, %s, %s,
    %s, %s, %s, %s,
    %s, %s, %s, %s
)
"""

# Convert DataFrame rows to list of tuples
data = [tuple(row) for row in df.values]

# Batch insert
cursor.executemany(insert_query, data)
connection.commit()

print("Data successfully loaded into Health_Data table!")


Data successfully loaded into Health_Data table!


In [17]:
# Validate Import
cursor.execute("SELECT COUNT(*) FROM Health_Data;")
print("Total Rows Inserted;", cursor.fetchone()[0])
cursor.execute("SELECT * FROM Health_Data LIMIT 5;")
print(cursor.fetchall())

Total Rows Inserted; 9949
(('PAT100000', 'Male', 76, 'Mumbai', 'Maharashtra', 'Pulmonology', datetime.date(2021, 1, 31), datetime.date(2021, 5, 2), 5, 'Semi-Private', 'Consultation', 162613, 'ICICI Lombard', Decimal('0.66'), 107324, 55289, 157246, 'Recovered', 1, 2021, 'Senior'), ('PAT100001', 'Female', 29, 'New Delhi', 'Delhi', 'Oncology', datetime.date(2023, 9, 1), datetime.date(2023, 5, 9), 4, 'General', 'Consultation', 146321, 'No Insurance', Decimal('0.00'), 0, 146321, 146321, 'Complication', 1, 2023, 'Adult'), ('PAT100003', 'Female', 74, 'Jaipur', 'Rajasthan', 'Emergency', datetime.date(2022, 3, 31), datetime.date(2022, 4, 4), 4, 'General', 'Therapy', 196772, 'No Insurance', Decimal('0.00'), 0, 196772, 196772, 'Recovered', 1, 2022, 'Senior'), ('PAT100005', 'Female', 35, 'Agra', 'Uttar Pradesh', 'Pulmonology', datetime.date(2025, 8, 31), datetime.date(2025, 5, 9), 5, 'General', 'Emergency Surgery', 95545, 'Tata AIG', Decimal('0.66'), 63059, 32486, 92392, 'Improved', 3, 2025, 'Adul

In [18]:
# close connection 
cursor.close()
connection.close()

In [19]:
# Check Total Rows in CSV 
import pandas as pd 
df= pd.read_csv("Cleaned_Health_Performance.csv")
print("CSV rows:", len(df))

CSV rows: 9949


In [22]:
# Export using python + pumysql 
import pymysql
import pandas as pd 

#connect to mysql 
connection = pymysql.connect(
    host="localhost",
    user="root",
    password="sravzlakna@8",
    database="health_performance"
)

# first query
query="""
SELECT department, SUM(Revenue_INR) AS Total_Revenue
FROM health_data
GROUP BY Department
ORDER BY Total_Revenue DESC
LIMIT 5; 
"""
# load results into pandas 
df=pd.read_sql(query,connection)

#export to CSV
df.to_csv("dept_revenue.csv", index=False)

print("Exported results to dept_revenue.csv")
connection.close()


Exported results to dept_revenue.csv


C:\Users\srava\AppData\Local\Temp\ipykernel_10388\3878609809.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(query,connection)


In [23]:
import pymysql
import pandas as pd

# Connect to MySQL
connection = pymysql.connect(
    host="localhost",
    user="root",
    password="sravzlakna@8",   
    database="health_performance"    
)

# Define queries with filenames
queries = [
    
    ("SELECT Insurance_Provider, AVG(Patient_Satisfaction_Rating) AS Avg_Rating \
      FROM Health_Data GROUP BY Insurance_Provider;", 
     "insurance_satisfaction.csv"),

    ("SELECT Outcome, COUNT(*) AS Count, \
             ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM Health_Data), 2) AS Percentage \
      FROM Health_Data GROUP BY Outcome;", 
     "treatment_success.csv"),

    ("SELECT Year, SUM(Revenue_INR) AS Yearly_Revenue \
      FROM Health_Data GROUP BY Year ORDER BY Year;", 
     "yearly_revenue.csv"),

    ("SELECT Treatment_Type, AVG(Length_of_Stay) AS Avg_Stay \
      FROM Health_Data GROUP BY Treatment_Type;", 
     "avg_stay.csv"),

    ("SELECT State, AVG(Patient_Pay_INR) AS Avg_Patient_Cost \
      FROM Health_Data GROUP BY State ORDER BY Avg_Patient_Cost DESC;", 
     "patient_cost.csv"),

    ("SELECT Age_Group, Outcome, COUNT(*) AS Count \
      FROM Health_Data GROUP BY Age_Group, Outcome;", 
     "agegroup_outcome.csv"),

    ("SELECT Insurance_Provider, SUM(Revenue_INR) AS Total_Revenue \
      FROM Health_Data GROUP BY Insurance_Provider;", 
     "insurance_revenue.csv"),

    ("SELECT City, AVG(Patient_Satisfaction_Rating) AS Avg_Rating \
      FROM Health_Data GROUP BY City HAVING AVG(Patient_Satisfaction_Rating) < 3;", 
     "low_satisfaction_cities.csv"),

    ("SELECT Room_Type, SUM(Revenue_INR) AS Total_Revenue \
      FROM Health_Data GROUP BY Room_Type;", 
     "roomtype_revenue.csv")
]

# Run queries and export results
for query, filename in queries:
    df = pd.read_sql(query, connection)
    df.to_csv(filename, index=False)
    print(f"Exported {filename}")

connection.close()


Exported insurance_satisfaction.csv
Exported treatment_success.csv
Exported yearly_revenue.csv
Exported avg_stay.csv
Exported patient_cost.csv
Exported agegroup_outcome.csv
Exported insurance_revenue.csv
Exported low_satisfaction_cities.csv
Exported roomtype_revenue.csv


C:\Users\srava\AppData\Local\Temp\ipykernel_10388\2765328252.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [25]:
df = pd.read_sql("SELECT * FROM health_data;", connection)
df.to_csv("health_data_Full.csv", index=False)


C:\Users\srava\AppData\Local\Temp\ipykernel_10388\3072334158.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM health_data;", connection)


DatabaseError: Execution failed on sql: SELECT * FROM health_data;
(0, '')
unable to rollback

In [26]:
import pymysql
import pandas as pd

connection = pymysql.connect(
    host="localhost",
    user="root",
    password="sravzlakna@8",
    database="health_performance"
)

cursor = connection.cursor()

cursor.execute("SELECT * FROM health_data;")
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

df = pd.DataFrame(rows, columns=columns)
df.to_csv("Health_Data_Full.csv", index=False)

print("Export complete! Rows exported:", len(df))

cursor.close()
connection.close()


Export complete! Rows exported: 9949


In [27]:
df.to_csv("Health_Data_Full.csv", index=False, encoding="utf-8")

In [28]:
df.to_csv("Health_Data_Full.csv", index=False, header=True, sep=",",encoding="utf-8")

In [29]:
check=pd.read_csv("Health_Data_Full.csv")
print(check.head())

  Patient_ID  Gender  Age       City          State   Department  \
0  PAT100000    Male   76     Mumbai    Maharashtra  Pulmonology   
1  PAT100001  Female   29  New Delhi          Delhi     Oncology   
2  PAT100003  Female   74     Jaipur      Rajasthan    Emergency   
3  PAT100005  Female   35       Agra  Uttar Pradesh  Pulmonology   
4  PAT100007    Male   66    Lucknow  Uttar Pradesh  Orthopedics   

  Admission_Date Discharge_Date  Length_of_Stay     Room_Type  ...  \
0     2021-01-31     2021-05-02               5  Semi-Private  ...   
1     2023-09-01     2023-05-09               4       General  ...   
2     2022-03-31     2022-04-04               4       General  ...   
3     2025-08-31     2025-05-09               5       General  ...   
4     2020-10-07     2020-12-10               5       Private  ...   

  Treatment_Cost_INR  Insurance_Provider Insurance_Coverage_Pct  \
0             162613       ICICI Lombard                   0.66   
1             146321        No Insur